# Poker Combinatorics

Since this part of the project was taking a long while we thought of documenting it.

Until we venture into calculating number of cases in different states we should get the basic initial combinatorics down, they may be used as a template for further calculations and it supposedly less complex.

In [3]:
from math import comb

In [4]:
# Royal Flushes
rf_cases = 4 * comb(47, 2)
rf_cases # Correct

4324

In [140]:
# Straight Flushes
sf_cases = 9 * 4 * comb(46, 2)
sf_cases # Correct

37260

In [6]:
# Flushes
fl_cases = 4 * (comb(13, 5) * comb(39, 2) + comb(13, 6) * comb(39, 1) + comb(13, 7))
fl_cases - rf_cases - sf_cases # Correct

4047644

In [135]:
# Straights
st_cases = 0

# Interpretation for the loop: We start from the best straight and we go down, 
# each following analysis has only one added restriction it cannot have the top 
# card from the previous straight, since then it would be the previous straight.
# Nevertheless, the loop is simply decorative, it could also be written as: 
# 1 * best straight cases + 9 * other straight cases
# Disclaimer: When we say random card, it means a random card that satisfies 
# the restrictions (the global restriction and of each case it's split into).
for i in range(10):
    # Cases with no pairs in the straight
    # 4 suits for each straight card * 2 random cards
    st_cases += 4**5 * comb(52 - 5*4 - 4 * min(i, 1), 2)
    # Cases with one pair in the straight
    # 4 suits for each straight card minus the paired one * paired card * suits for the pair * random card
    st_cases += 4**4 * comb(5, 1) * comb(4, 2) * comb(52 - 5*4 - 4 * min(i, 1), 1)
    # Cases with two pairs in the straight
    # 4 suits for each straight card minus the paired ones * 2 paired cards * suits for the pairs
    st_cases += 4**3 * comb(5, 2) * comb(4, 2)**2
    # Cases with a three of a kind in the straight
    # 4 suits for each straight card minus the three of a kind * three of a kind * suits for the other 2 cards
    st_cases += 4**4 * comb(5, 1) * comb(4, 3)
    
    # 4 suits * 2 random cards (royal and flushed straights)
    # st_cases -= 4 * comb(47 - min(i, 1), 2)
    # If we remove that we would get all the straights without straight flushes
    # or royal straights, but there are some cases flushes can form without 
    # forming a straight flush or royal straight.
    # If we don't discount the flushes, we've seen the number is correct from 
    # different sources. 
    
    # See the top answer by "Daniel Mathias"
    # Requires a couple of operations to add the flushes back in and remove 
    # royal and flushed straights, but yields this number: (Also, it calculates 
    # correctly the true number of straights, but their scheme lacks any
    # adaptability potential towards calculating it in different phases of the 
    # game, some numbers are extremely specific to the situation)
    # https://math.stackexchange.com/questions/3084135/calculating-combinations-of-a-straight-in-texas-hold-em#:~:text=With%20no%20pair%2C%20we%20can,th%20card%20(162%20possible).
    
    # Original poster and comment by "okzoomer" both have the number we obtained
    # They also comment on the impossibility of adjusting by a factor, since 
    # getting a straight is not independent of getting a flush (it makes it more 
    # likely due to at least 5 cards having different ranks) (it also makes it 
    # less likely due to royal and flushed straights not being considered 
    # straights, but that is easily adjustable).
    # https://math.stackexchange.com/questions/4728513/how-do-i-calculate-the-probability-of-getting-a-5-card-straight-in-7-card-poker?newreg=6c83afc136bf43bcad3c2ab79c7a3767
    
    
# We need to remove cases with flushes
# Test cases
# A K Q J T + +
# 1 2 3 4 5 + +
for i in range(10):
    # Cases with no pairs in the straight
    st_cases -= 4 * (
        # Preamble: All cards from the straight have the flushed suit
        # 2 random cards (the straight is already flushed)
        # Total flushed cards: 5 (+ 2 optional)
        comb(52 - 5*4 - 4 * min(i, 1), 2)
        # Preamble: 1 card from the straight does not have the flushed suit
        # flushed extra card * random card * unflushed card * 3 suits for the unflushed card
        # Total flushed cards: 4 + 1 (+ 1 optional)
        + comb(13 - 5 - min(i, 1), 1) * comb(52 - 5*4 - 4 * min(i, 1) - 1, 1) * comb(5, 1) * 3
        # Preamble: 2 cards from the straight do not have the flushed suit
        # 2 flushed extra cards * 2 unflushed cards * 3 suits for the unflushed cards
        # Total flushed cards: 3 + 2
        + comb(13 - 5 - min(i, 1), 2) * comb(5, 2) * 3**2
    )
    
    # Cases with one pair in the straight
    st_cases -= 4 * (
        # Preamble: All cards from the straight have the flushed suit
        # random card * paired card * 3 suits for that paired card (the entire straight is flushed)
        # Total flushed cards: 5 (+ 1 optional)
        comb(52 - 5*4 - 4 * min(i, 1), 1) * comb(5, 1) * 3
        # Preamble: 1 card from the straight does not have the flushed suit
        # flushed extra card * paired card * (3 suits for the paired card * 3 suits for the unflushed card or 3 suits choose 2 [unflushed is paired])
        # Total flushed cards: 4 + 1
        + comb(13 - 5 - min(i, 1), 1) * comb(5, 1) * (comb(4, 1) * 3**2 + comb(3, 2)) # 4 + 1
    )
    
    # Cases with two pairs in the straight
    st_cases -= 4 * (
        # Preamble: All cards from the straight have the flushed suit
        # 2 paired cards * 3 suits for each paired cards
        comb(5, 2) * 3 * 3
    )
    
    # Cases with a three of a kind in the straight
    st_cases -= 4 * (
        # Preamble: All cards from the straight have the flushed suit
        # 1 three of a kind * 3 chose 2 suits for the other 2 cards
        comb(5, 1) * comb(3, 2)
    )
    
st_cases 
# It is wrong :(
# We should get: 6180020 (source: Wikipedia and our previous sources)

6167000

In [139]:
(6180020 - 6167000) / comb(52, 7) # It is a very small fraction but it could be determinant for the following phases like the river

9.732064746484946e-05

In [141]:
(6180020 - 6167000) / 6180020 # 2% of migdiagnozed cases might be a problem

0.00210678929841651

In [ ]:
# Until this problem is solved, we cannot continue with the complex equity calculator